In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Loading the dataset
#data1 = pd.read_csv('MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv')
data2 = pd.read_csv('MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv')
data3 = pd.read_csv('MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv')
data4 = pd.read_csv('MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
data5 = pd.read_csv('MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
#data6 = pd.read_csv('MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv')
data7 = pd.read_csv('MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
data8 = pd.read_csv('MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [ ]:
data_list = [data2, data3, data4, data5, data7, data8]

print('Data dimensions: ')
for i, data in enumerate(data_list, start = 1):
  rows, cols = data.shape
  print(f'Data{i} -> {rows} rows, {cols} columns')

In [ ]:
df = pd.concat(data_list)
rows, cols = df.shape

print('New dimension:')
print(f'Number of rows: {rows}')
print(f'Number of columns: {cols}')
print(f'Total cells: {rows * cols}')

In [ ]:
# Deleting dataframes after concating to save memory
for d in data_list: del d

In [ ]:
# Renaming the columns by removing leading/trailing whitespace
col_names = {col: col.strip() for col in df.columns}
df.rename(columns = col_names, inplace = True)

In [ ]:
print(df.columns)

In [ ]:
# Separate features from labels
X = df.drop(columns=['Label'])
y = df['Label']

In [ ]:
num_splits = 100
split_size = len(df) // num_splits

dfs = []
start = 0
for i in range(num_splits):
    end = start + split_size
    df_slice = df.iloc[start:end].copy()  # copy the slice to avoid reindexing issues

    # Replace infinities in the slice
    df_slice.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Drop or fill NaNs
    df_slice.dropna(inplace=True)

    dfs.append(df_slice)
    start = end

# Last slice for any leftover rows
if end < len(df):
    df_slice = df.iloc[end:].copy()
    df_slice.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_slice.dropna(inplace=True)
    dfs.append(df_slice)

df = pd.concat(dfs, ignore_index=True)
print("DataFrame shape after cleaning:", df.shape)


In [ ]:
# Replace +∞, -∞ with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows that have become NaN
df.dropna(inplace=True)

# Now separate X and y again
X = df.drop(columns=['Label'])
y = df['Label']

# Then you can do your scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# 70% train, 15% val, 15% test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.15, 
    stratify=y, 
    random_state=42
)

# Now split train_val into train and validation
val_ratio = 0.15 / 0.85  # roughly 0.1765
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=val_ratio, 
    stratify=y_train_val, 
    random_state=42
)

print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Justification for 70-15-15 split:

70% Training: Enough data to learn robust patterns.

15% Validation: A separate set for hyperparameter tuning (e.g., learning rate, noise multiplier for DP, etc.).

15% Test: Kept strictly for final evaluation. This prevents overfitting to the validation set and provides an unbiased measure of performance.

In [ ]:
#%pip install tensorflow-privacy

In [ ]:
#%pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [ ]:
# Hyperparameters
learning_rate   = 0.01
batch_size      = 256
microbatch_size = 64
epochs          = 5
noise_multiplier = 1.1
l2_norm_clip    = 1.0
delta           = 1e-5  # Typical DP delta

def create_model(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # for binary classification
    ])
    return model

model = create_model(X_train.shape[1])

dp_optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=microbatch_size,
    learning_rate=learning_rate
)

model.compile(
    optimizer=dp_optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    verbose=1
)

# Compute approximate epsilon
eps, _ = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=len(X_train),
    batch_size=batch_size,
    noise_multiplier=noise_multiplier,
    epochs=epochs,
    delta=delta
)
print(f"For delta={delta}, the estimated epsilon is: {eps:.2f}")


In [ ]:
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score, 
    accuracy_score, roc_auc_score
)

y_prob = model.predict(X_test).ravel()
y_pred = (y_prob >= 0.5).astype(int)

acc  = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='binary')
rec  = recall_score(y_test, y_pred, average='binary')
f1   = f1_score(y_test, y_pred, average='binary')
auc  = roc_auc_score(y_test, y_prob)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-Score:", f1)
print("AUC-ROC:", auc)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


Interpretation of Confusion Matrix

TP: Attack traffic correctly labeled “attack”.

TN: Benign traffic correctly labeled “benign”.

FP: Benign mislabeled as attack (false alarms).

FN: Attacks mislabeled as benign (missed detections).

Privacy (ε) vs. Accuracy Trade-Off

A higher noise_multiplier → better privacy (lower ε) but generally lower accuracy.

A lower noise_multiplier → weaker privacy (larger ε) but higher accuracy.

Goal: Find a balance that meets your organization’s regulatory/privacy needs while maintaining high detection performance.

In [ ]:
NUM_CLIENTS = 5
client_data = []
total_samples = len(X_train)
samples_per_client = total_samples // NUM_CLIENTS

start = 0
for i in range(NUM_CLIENTS):
    end = start + samples_per_client
    X_c = X_train[start:end]
    y_c = y_train[start:end]
    client_data.append((X_c, y_c))
    start = end

# You might have leftover samples if total_samples is not divisible by NUM_CLIENTS.
# Those leftover samples can go to the last client or be distributed among them.


In [ ]:
#%pip install tensorflow_federated


In [ ]:
import tensorflow_federated as tff
import tensorflow as tf

def create_tf_dataset_for_client(data_tuple):
    X_c, y_c = data_tuple
    ds = tf.data.Dataset.from_tensor_slices((X_c, y_c))
    ds = ds.shuffle(buffer_size=100).batch(32)
    return ds

federated_train_data = [create_tf_dataset_for_client(cd) for cd in client_data]

In [ ]:
def model_fn():
    keras_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

In [ ]:
iterative_process = tff.learning.build_federated_averaging_process(model_fn)
state = iterative_process.initialize()

NUM_ROUNDS = 5
for round_num in range(1, NUM_ROUNDS+1):
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f"Round {round_num} metrics: {metrics}")

In [ ]:
from tensorflow_federated.python.aggregators import differential_privacy

dp_factory = differential_privacy.DifferentiallyPrivateFactory.gaussian_fixed(
    noise_multiplier=1.0,
    clients_per_round=NUM_CLIENTS,
    clipped_count_budget=10,  # or as desired
    l2_norm_clip=1.0
)

dp_iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    model_aggregator=dp_factory.create(
        value_type=iterative_process.get_model_weights(state).trainable
    )
)

dp_state = dp_iterative_process.initialize()

for round_num in range(1, NUM_ROUNDS+1):
    dp_state, dp_metrics = dp_iterative_process.next(dp_state, federated_train_data)
    print(f"Round {round_num} DP metrics: {dp_metrics}")

In [ ]:
federated_weights = dp_iterative_process.get_model_weights(dp_state)
model_fed = create_model(X_train.shape[1])  # same architecture as model_fn
federated_weights.assign_weights_to(model_fed)

y_prob_fed = model_fed.predict(X_test).ravel()
y_pred_fed = (y_prob_fed >= 0.5).astype(int)

acc_fed = accuracy_score(y_test, y_pred_fed)
print("Federated model accuracy on test set:", acc_fed)

Comparison with Centralized Model

2.2 Adversarial Analysis and Defense

Model Poisoning: A malicious client uploads gradients that skew or degrade the model.

Defense: Use robust aggregators (e.g., multi-Krum, outlier detection, or differential privacy** with clipping**).

Inference Attacks: Attackers try to infer details about a client’s data from global model parameters.

Defense: User-level differential privacy ensures that updates are clipped and noised, reducing the potential for inference.

Backdoor Attacks: Attackers embed triggers in the model.

Defense: Periodic validation against known safe data, outlier detection in gradient updates, or specialized defensive techniques (e.g., verifiable model updates).